# CVRP Solver: Local Search + VND + Simulated Annealing + Tabu Search

This notebook implements a hybrid metaheuristic algorithm for solving Capacitated Vehicle Routing Problems (CVRP) **without time windows**.

## Algorithm Components:
1. **Initial Solution**: Nearest Neighbor heuristic
2. **Local Search**: Improvement operators
3. **VND (Variable Neighborhood Descent)**: Systematic neighborhood exploration
4. **Simulated Annealing**: Accept worse solutions with decreasing probability
5. **Tabu Search**: Memory structure to avoid cycling

## Goal:
Find solutions within **7% gap** from the optimal solution.

In [19]:
# Import required libraries
import numpy as np
import vrplib
import yaml
import random
import math
import time
import os
from collections import deque
from copy import deepcopy
from typing import List, Dict, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [20]:
# Load configuration from YAML file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(yaml.dump(config, default_flow_style=False))

Configuration loaded:
general:
  random_seed: 42
  time_limit_seconds: 300
  verbose: true
initial_solution:
  method: nearest_neighbor
  randomness: 0.1
local_search:
  max_iterations: 1000
  max_iterations_without_improvement: 200
quality:
  penalty_capacity_violation: 10000
  target_gap_percentage: 7.0
simulated_annealing:
  alpha: 0.95
  final_temperature: 0.1
  initial_temperature: 1000.0
  iterations_per_temperature: 100
tabu_search:
  aspiration_enabled: true
  tabu_tenure: 20
  tabu_tenure_random_range: 10
vnd:
  max_iterations_without_improvement: 50
  neighborhoods:
  - swap
  - relocate
  - two_opt
  - cross_exchange



In [21]:
# Set random seed for reproducibility
random.seed(config['general']['random_seed'])
np.random.seed(config['general']['random_seed'])

print(f"Random seed set to: {config['general']['random_seed']}")

Random seed set to: 42


## Data Structures and Helper Functions

In [22]:
class CVRPSolution:
    """Represents a CVRP solution with routes and cost."""
    
    def __init__(self, routes: List[List[int]], distance_matrix: np.ndarray):
        self.routes = routes
        self.distance_matrix = distance_matrix
        self.cost = self.calculate_cost()
    
    def calculate_cost(self) -> float:
        """Calculate total cost of the solution."""
        total_cost = 0
        for route in self.routes:
            if len(route) == 0:
                continue
            # Add distance from depot (0) to first customer
            total_cost += self.distance_matrix[0, route[0]]
            # Add distances between customers
            for i in range(len(route) - 1):
                total_cost += self.distance_matrix[route[i], route[i+1]]
            # Add distance from last customer back to depot
            total_cost += self.distance_matrix[route[-1], 0]
        return total_cost
    
    def update_cost(self):
        """Recalculate cost after modification."""
        self.cost = self.calculate_cost()
    
    def is_feasible(self, demands: np.ndarray, capacity: int) -> bool:
        """Check if solution satisfies capacity constraints."""
        for route in self.routes:
            route_demand = sum(demands[customer] for customer in route)
            if route_demand > capacity:
                return False
        return True
    
    def copy(self):
        """Create a deep copy of the solution."""
        return CVRPSolution([route.copy() for route in self.routes], self.distance_matrix)

print("CVRPSolution class defined!")

CVRPSolution class defined!


In [23]:
def calculate_distance_matrix(instance: Dict) -> np.ndarray:
    """Calculate distance matrix from instance data."""
    if 'edge_weight' in instance:
        # Distance matrix already provided
        return instance['edge_weight']
    elif 'node_coord' in instance:
        # Calculate Euclidean distances
        coords = np.array(instance['node_coord'])
        n = len(coords)
        dist_matrix = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i != j:
                    dist = np.sqrt(np.sum((coords[i] - coords[j])**2))
                    dist_matrix[i, j] = dist
        return dist_matrix
    else:
        raise ValueError("Instance must have either 'edge_weight' or 'node_coord'")

print("Distance matrix calculation function defined!")

Distance matrix calculation function defined!


## Initial Solution Construction

In [24]:
def nearest_neighbor_solution(instance: Dict, distance_matrix: np.ndarray) -> CVRPSolution:
    """Create initial solution using nearest neighbor heuristic."""
    n_customers = instance['dimension'] - 1  # Exclude depot
    capacity = instance['capacity']
    demands = np.array(instance['demand'])
    
    unvisited = set(range(1, n_customers + 1))
    routes = []
    
    while unvisited:
        route = []
        current_load = 0
        current_node = 0  # Start from depot
        
        while True:
            # Find nearest feasible customer
            best_customer = None
            best_distance = float('inf')
            
            for customer in unvisited:
                if current_load + demands[customer] <= capacity:
                    dist = distance_matrix[current_node, customer]
                    # Add randomness based on config
                    randomness = config['initial_solution']['randomness']
                    dist *= (1 + randomness * random.random())
                    
                    if dist < best_distance:
                        best_distance = dist
                        best_customer = customer
            
            if best_customer is None:
                break  # No feasible customer, close route
            
            route.append(best_customer)
            current_load += demands[best_customer]
            current_node = best_customer
            unvisited.remove(best_customer)
        
        if route:
            routes.append(route)
    
    return CVRPSolution(routes, distance_matrix)

print("Nearest neighbor solution function defined!")

Nearest neighbor solution function defined!


## Neighborhood Operators

In [25]:
def swap_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Swap two customers between different routes."""
    if len(solution.routes) < 2:
        return None
    
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            
            if len(route_i) == 0 or len(route_j) == 0:
                continue
            
            for pos_i in range(len(route_i)):
                for pos_j in range(len(route_j)):
                    # Create new solution
                    new_solution = solution.copy()
                    
                    # Swap customers
                    new_solution.routes[i][pos_i], new_solution.routes[j][pos_j] = \
                        new_solution.routes[j][pos_j], new_solution.routes[i][pos_i]
                    
                    # Check feasibility
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Swap operator defined!")

Swap operator defined!


In [26]:
def relocate_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Relocate a customer from one route to another."""
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        route_i = solution.routes[i]
        if len(route_i) == 0:
            continue
            
        for pos_i in range(len(route_i)):
            customer = route_i[pos_i]
            
            for j in range(len(solution.routes)):
                if i == j:
                    continue
                
                for pos_j in range(len(solution.routes[j]) + 1):
                    # Create new solution
                    new_solution = solution.copy()
                    
                    # Remove customer from route i
                    removed = new_solution.routes[i].pop(pos_i)
                    
                    # Insert into route j
                    new_solution.routes[j].insert(pos_j, removed)
                    
                    # Check feasibility
                    if new_solution.is_feasible(demands, capacity):
                        new_solution.update_cost()
                        if new_solution.cost < best_cost:
                            best_cost = new_solution.cost
                            best_solution = new_solution
    
    return best_solution

print("Relocate operator defined!")

Relocate operator defined!


In [27]:
def two_opt_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Apply 2-opt improvement within each route."""
    best_solution = None
    best_cost = solution.cost
    
    for route_idx in range(len(solution.routes)):
        route = solution.routes[route_idx]
        if len(route) < 2:
            continue
        
        for i in range(len(route) - 1):
            for j in range(i + 1, len(route)):
                # Create new solution
                new_solution = solution.copy()
                
                # Reverse segment between i and j
                new_solution.routes[route_idx][i:j+1] = reversed(new_solution.routes[route_idx][i:j+1])
                
                # No need to check feasibility (2-opt preserves route assignment)
                new_solution.update_cost()
                if new_solution.cost < best_cost:
                    best_cost = new_solution.cost
                    best_solution = new_solution
    
    return best_solution

print("2-opt operator defined!")

2-opt operator defined!


In [28]:
def cross_exchange_operator(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> Optional[CVRPSolution]:
    """Exchange segments between two routes."""
    if len(solution.routes) < 2:
        return None
    
    best_solution = None
    best_cost = solution.cost
    
    for i in range(len(solution.routes)):
        for j in range(i + 1, len(solution.routes)):
            route_i = solution.routes[i]
            route_j = solution.routes[j]
            
            if len(route_i) < 2 or len(route_j) < 2:
                continue
            
            # Try exchanging segments of length 1 or 2
            for seg_len in [1, 2]:
                for pos_i in range(len(route_i) - seg_len + 1):
                    for pos_j in range(len(route_j) - seg_len + 1):
                        # Create new solution
                        new_solution = solution.copy()
                        
                        # Extract segments
                        seg_i = new_solution.routes[i][pos_i:pos_i+seg_len]
                        seg_j = new_solution.routes[j][pos_j:pos_j+seg_len]
                        
                        # Exchange segments
                        new_solution.routes[i][pos_i:pos_i+seg_len] = seg_j
                        new_solution.routes[j][pos_j:pos_j+seg_len] = seg_i
                        
                        # Check feasibility
                        if new_solution.is_feasible(demands, capacity):
                            new_solution.update_cost()
                            if new_solution.cost < best_cost:
                                best_cost = new_solution.cost
                                best_solution = new_solution
    
    return best_solution

print("Cross-exchange operator defined!")

Cross-exchange operator defined!


## Variable Neighborhood Descent (VND)

In [29]:
def vnd(solution: CVRPSolution, demands: np.ndarray, capacity: int) -> CVRPSolution:
    """Variable Neighborhood Descent algorithm."""
    neighborhoods = {
        'swap': swap_operator,
        'relocate': relocate_operator,
        'two_opt': two_opt_operator,
        'cross_exchange': cross_exchange_operator
    }
    
    neighborhood_order = config['vnd']['neighborhoods']
    max_no_improve = config['vnd']['max_iterations_without_improvement']
    
    current_solution = solution
    k = 0  # Neighborhood index
    no_improve_count = 0
    
    while k < len(neighborhood_order) and no_improve_count < max_no_improve:
        neighborhood_name = neighborhood_order[k]
        operator = neighborhoods[neighborhood_name]
        
        # Apply operator
        new_solution = operator(current_solution, demands, capacity)
        
        if new_solution is not None and new_solution.cost < current_solution.cost:
            current_solution = new_solution
            k = 0  # Reset to first neighborhood
            no_improve_count = 0
        else:
            k += 1  # Try next neighborhood
            no_improve_count += 1
    
    return current_solution

print("VND function defined!")

VND function defined!


## Tabu Search

In [30]:
class TabuList:
    """Tabu list for tracking forbidden moves."""
    
    def __init__(self, tenure: int):
        self.tenure = tenure
        self.tabu_dict = {}  # (move_description) -> iteration_when_expires
        self.current_iteration = 0
    
    def add(self, move: Tuple, tenure_variation: int = 0):
        """Add a move to the tabu list."""
        actual_tenure = self.tenure + random.randint(-tenure_variation, tenure_variation)
        self.tabu_dict[move] = self.current_iteration + actual_tenure
    
    def is_tabu(self, move: Tuple) -> bool:
        """Check if a move is tabu."""
        if move not in self.tabu_dict:
            return False
        return self.tabu_dict[move] > self.current_iteration
    
    def increment_iteration(self):
        """Move to next iteration and clean expired moves."""
        self.current_iteration += 1
        # Remove expired moves
        expired = [move for move, expiration in self.tabu_dict.items() 
                   if expiration <= self.current_iteration]
        for move in expired:
            del self.tabu_dict[move]

print("TabuList class defined!")

TabuList class defined!


## Simulated Annealing with Tabu Search

In [31]:
def acceptance_probability(current_cost: float, new_cost: float, temperature: float) -> float:
    """Calculate acceptance probability for SA."""
    if new_cost < current_cost:
        return 1.0
    return math.exp((current_cost - new_cost) / temperature)

def simulated_annealing_with_tabu(
    initial_solution: CVRPSolution, 
    demands: np.ndarray, 
    capacity: int,
    time_limit: float = None
) -> Tuple[CVRPSolution, List[float]]:
    """Hybrid Simulated Annealing with Tabu Search and VND."""
    
    # Parameters from config
    temp = config['simulated_annealing']['initial_temperature']
    final_temp = config['simulated_annealing']['final_temperature']
    alpha = config['simulated_annealing']['alpha']
    iterations_per_temp = config['simulated_annealing']['iterations_per_temperature']
    
    tabu_tenure = config['tabu_search']['tabu_tenure']
    tabu_tenure_variation = config['tabu_search']['tabu_tenure_random_range']
    aspiration_enabled = config['tabu_search']['aspiration_enabled']
    
    max_iterations = config['local_search']['max_iterations']
    max_no_improve = config['local_search']['max_iterations_without_improvement']
    
    # Initialize
    current_solution = initial_solution
    best_solution = current_solution.copy()
    tabu_list = TabuList(tabu_tenure)
    
    cost_history = [best_solution.cost]
    no_improve_count = 0
    total_iterations = 0
    
    start_time = time.time()
    
    # Neighborhood operators
    operators = [swap_operator, relocate_operator, two_opt_operator, cross_exchange_operator]
    
    while temp > final_temp and total_iterations < max_iterations:
        # Check time limit
        if time_limit and (time.time() - start_time) > time_limit:
            break
        
        if no_improve_count >= max_no_improve:
            break
        
        for _ in range(iterations_per_temp):
            # Apply VND periodically
            if total_iterations % 50 == 0:
                current_solution = vnd(current_solution, demands, capacity)
            
            # Select random operator
            operator = random.choice(operators)
            new_solution = operator(current_solution, demands, capacity)
            
            if new_solution is None:
                continue
            
            # Create move identifier (simplified)
            move_id = (operator.__name__, hash(str(new_solution.routes)))
            
            # Check tabu status
            is_tabu = tabu_list.is_tabu(move_id)
            
            # Aspiration criterion: accept tabu move if it's better than best
            aspiration = aspiration_enabled and new_solution.cost < best_solution.cost
            
            # Decide acceptance
            if (not is_tabu or aspiration):
                # SA acceptance criterion
                if random.random() < acceptance_probability(current_solution.cost, new_solution.cost, temp):
                    current_solution = new_solution
                    tabu_list.add(move_id, tabu_tenure_variation)
                    
                    # Update best solution
                    if current_solution.cost < best_solution.cost:
                        best_solution = current_solution.copy()
                        cost_history.append(best_solution.cost)
                        no_improve_count = 0
                        if config['general']['verbose']:
                            print(f"  New best: {best_solution.cost:.2f} at iteration {total_iterations}")
                    else:
                        no_improve_count += 1
            
            tabu_list.increment_iteration()
            total_iterations += 1
        
        # Cool down
        temp *= alpha
    
    return best_solution, cost_history

print("Simulated Annealing with Tabu Search function defined!")

Simulated Annealing with Tabu Search function defined!


## Main Solver Function

In [32]:
def solve_cvrp(instance_path: str) -> Dict:
    """Solve a CVRP instance and return results."""
    
    print(f"\n{'='*60}")
    print(f"Solving: {os.path.basename(instance_path)}")
    print(f"{'='*60}")
    
    # Read instance
    try:
        instance = vrplib.read_instance(instance_path)
    except:
        # Try Solomon format
        instance = vrplib.read_instance(instance_path, instance_format='solomon')
    
    # Calculate distance matrix
    distance_matrix = calculate_distance_matrix(instance)
    demands = np.array(instance['demand'])
    capacity = instance['capacity']
    
    print(f"Dimension: {instance['dimension']} nodes")
    print(f"Capacity: {capacity}")
    
    # Create initial solution
    print("\nCreating initial solution...")
    initial_solution = nearest_neighbor_solution(instance, distance_matrix)
    print(f"Initial cost: {initial_solution.cost:.2f}")
    print(f"Initial routes: {len(initial_solution.routes)}")
    
    # Apply VND for quick improvement
    print("\nApplying VND...")
    improved_solution = vnd(initial_solution, demands, capacity)
    print(f"After VND cost: {improved_solution.cost:.2f}")
    
    # Apply SA with Tabu
    print("\nApplying Simulated Annealing with Tabu Search...")
    time_limit = config['general']['time_limit_seconds']
    start_time = time.time()
    
    final_solution, cost_history = simulated_annealing_with_tabu(
        improved_solution, demands, capacity, time_limit
    )
    
    elapsed_time = time.time() - start_time
    
    print(f"\nFinal cost: {final_solution.cost:.2f}")
    print(f"Final routes: {len(final_solution.routes)}")
    print(f"Time elapsed: {elapsed_time:.2f} seconds")
    
    # Read optimal solution if available
    optimal_cost = None
    gap_percentage = None
    
    sol_path = instance_path.replace('.vrp', '.sol').replace('.txt', '.sol')
    if os.path.exists(sol_path):
        try:
            optimal_solution = vrplib.read_solution(sol_path)
            optimal_cost = optimal_solution['cost']
            gap_percentage = ((final_solution.cost - optimal_cost) / optimal_cost) * 100
            
            print(f"\nOptimal cost: {optimal_cost:.2f}")
            print(f"Gap: {gap_percentage:.2f}%")
            
            if gap_percentage <= config['quality']['target_gap_percentage']:
                print(f"✓ Solution within target gap of {config['quality']['target_gap_percentage']}%")
            else:
                print(f"✗ Solution exceeds target gap of {config['quality']['target_gap_percentage']}%")
        except Exception as e:
            print(f"Could not read optimal solution: {e}")
    
    return {
        'instance_name': os.path.basename(instance_path),
        'solution': final_solution,
        'cost': final_solution.cost,
        'optimal_cost': optimal_cost,
        'gap_percentage': gap_percentage,
        'n_routes': len(final_solution.routes),
        'time_seconds': elapsed_time,
        'cost_history': cost_history
    }

print("Main solver function defined!")

Main solver function defined!


## Save Solution to File

In [33]:
def save_solution(result: Dict, output_dir: str = 'solutions'):
    """Save solution to a file in VRPLIB format."""
    os.makedirs(output_dir, exist_ok=True)
    
    instance_name = result['instance_name'].replace('.vrp', '').replace('.txt', '')
    output_path = os.path.join(output_dir, f"{instance_name}_computed.sol")
    
    # Prepare routes (add 1 to convert from 0-indexed to 1-indexed, excluding depot)
    routes = result['solution'].routes
    
    # Write solution
    with open(output_path, 'w') as f:
        for idx, route in enumerate(routes, 1):
            route_str = ' '.join(map(str, route))
            f.write(f"Route #{idx}: {route_str}\n")
        f.write(f"Cost {result['cost']:.0f}\n")
    
    print(f"Solution saved to: {output_path}")
    return output_path

print("Save solution function defined!")

Save solution function defined!


## Test on Sample Instances

In [34]:
# Get list of CVRP instances (exclude Solomon instances which have time windows)
data_dir = 'data'
cvrp_instances = [
    os.path.join(data_dir, f) for f in os.listdir(data_dir) 
    if f.endswith('.vrp')
]

print(f"Found {len(cvrp_instances)} CVRP instances:")
for inst in cvrp_instances[:10]:  # Show first 10
    print(f"  - {os.path.basename(inst)}")
if len(cvrp_instances) > 10:
    print(f"  ... and {len(cvrp_instances) - 10} more")

Found 12 CVRP instances:
  - A-n32-k5.vrp
  - B-n31-k5.vrp
  - CMT6.vrp
  - E-n13-k4.vrp
  - F-n72-k4.vrp
  - Golden_1.vrp
  - Li_21.vrp
  - M-n101-k10.vrp
  - ORTEC-n242-k12.vrp
  - P-n16-k8.vrp
  ... and 2 more


In [35]:
# Solve a small instance as a test
test_instance = 'data/E-n13-k4.vrp'
result = solve_cvrp(test_instance)

# Save solution
save_solution(result)


Solving: E-n13-k4.vrp
Dimension: 13 nodes
Capacity: 6000

Creating initial solution...
Initial cost: 352.00
Initial routes: 4

Applying VND...
After VND cost: 290.00

Applying Simulated Annealing with Tabu Search...



Final cost: 290.00
Final routes: 4
Time elapsed: 40.70 seconds

Optimal cost: 247.00
Gap: 17.41%
✗ Solution exceeds target gap of 7.0%
Solution saved to: solutions/E-n13-k4_computed.sol


'solutions/E-n13-k4_computed.sol'

## Solve Multiple Instances

In [36]:
# Select a subset of instances for testing
selected_instances = [
    'data/A-n32-k5.vrp',
    'data/B-n31-k5.vrp',
    'data/CMT6.vrp'
]

# Filter to only existing instances
selected_instances = [inst for inst in selected_instances if os.path.exists(inst)]

results = []

for instance_path in selected_instances:
    try:
        result = solve_cvrp(instance_path)
        save_solution(result)
        results.append(result)
    except Exception as e:
        print(f"Error solving {instance_path}: {e}")
        continue

print(f"\n\nCompleted solving {len(results)} instances!")


Solving: A-n32-k5.vrp
Dimension: 32 nodes
Capacity: 100

Creating initial solution...
Initial cost: 1138.64
Initial routes: 5

Applying VND...
After VND cost: 797.45

Applying Simulated Annealing with Tabu Search...

Final cost: 797.45
Final routes: 5
Time elapsed: 300.66 seconds

Optimal cost: 784.00
Gap: 1.72%
✓ Solution within target gap of 7.0%
Solution saved to: solutions/A-n32-k5_computed.sol

Solving: B-n31-k5.vrp
Dimension: 31 nodes
Capacity: 100

Creating initial solution...
Initial cost: 899.13
Initial routes: 5

Applying VND...
After VND cost: 681.92

Applying Simulated Annealing with Tabu Search...

Final cost: 681.92
Final routes: 5
Time elapsed: 301.67 seconds

Optimal cost: 672.00
Gap: 1.48%
✓ Solution within target gap of 7.0%
Solution saved to: solutions/B-n31-k5_computed.sol

Solving: CMT6.vrp
Dimension: 51 nodes
Capacity: 160

Creating initial solution...
Initial cost: 699.10
Initial routes: 5

Applying VND...
After VND cost: 578.78

Applying Simulated Annealing wit

## Summary Statistics

In [37]:
# Create summary report
import pandas as pd

summary_data = []
for result in results:
    summary_data.append({
        'Instance': result['instance_name'],
        'Computed Cost': f"{result['cost']:.2f}",
        'Optimal Cost': f"{result['optimal_cost']:.2f}" if result['optimal_cost'] else 'N/A',
        'Gap (%)': f"{result['gap_percentage']:.2f}" if result['gap_percentage'] is not None else 'N/A',
        'Routes': result['n_routes'],
        'Time (s)': f"{result['time_seconds']:.2f}"
    })

df_summary = pd.DataFrame(summary_data)
print("\nSummary of Results:")
print(df_summary.to_string(index=False))

# Calculate statistics for instances with known optimal
gaps = [r['gap_percentage'] for r in results if r['gap_percentage'] is not None]
if gaps:
    print(f"\n\nGap Statistics:")
    print(f"  Average gap: {np.mean(gaps):.2f}%")
    print(f"  Median gap: {np.median(gaps):.2f}%")
    print(f"  Min gap: {np.min(gaps):.2f}%")
    print(f"  Max gap: {np.max(gaps):.2f}%")
    print(f"  Instances within {config['quality']['target_gap_percentage']}%: {sum(g <= config['quality']['target_gap_percentage'] for g in gaps)}/{len(gaps)}")


Summary of Results:
    Instance Computed Cost Optimal Cost Gap (%)  Routes Time (s)
A-n32-k5.vrp        797.45       784.00    1.72       5   300.66
B-n31-k5.vrp        681.92       672.00    1.48       5   301.67
    CMT6.vrp        578.78       555.43    4.20       5   308.00


Gap Statistics:
  Average gap: 2.47%
  Median gap: 1.72%
  Min gap: 1.48%
  Max gap: 4.20%
  Instances within 7.0%: 3/3


In [38]:
# Save summary to CSV
summary_path = 'solutions/summary_results.csv'
df_summary.to_csv(summary_path, index=False)
print(f"Summary saved to: {summary_path}")

Summary saved to: solutions/summary_results.csv
